In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the historical stock price data
data = pd.read_csv('stock_data.csv')
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices)

# Defining the number of previous days to consider for prediction
look_back = 20

# Creating input and target sequences
X = []
y = []
for i in range(look_back, len(prices_scaled)):
    X.append(prices_scaled[i - look_back:i, 0])
    y.append(prices_scaled[i, 0])
X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Reshape the input data for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Creating an LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, batch_size=32, epochs=50)

# Making predictions on the test data
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Ploting the actual and predicted stock prices
plt.figure(figsize=(12, 6))
plt.plot(data['Date'][split_index+look_back:], prices[split_index+look_back:], label='Actual Price')
plt.plot(data['Date'][split_index+look_back:], predictions, label='Predicted Price')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.title('Stock Price Prediction using RNN')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'